# **the main stocks class**

This Python program defines a Stocks Portfolio Management System, allowing users to buy and sell stocks while tracking their available funds. The StockItem class represents individual stocks with a name and quantity, while the Stocks class manages the overall portfolio. The buy method ensures that stocks can only be purchased if there are enough funds, either adding a new stock to the portfolio or increasing the quantity of an existing one. The sell method allows selling stocks if they are owned, reducing the quantity and removing the stock if all shares are sold. The display_portfolio method provides an overview of available funds and owned stocks. This system efficiently tracks transactions and prevents invalid operations, ensuring a smooth stock trading experience.

In [ ]:
class StockItem:
    """
    Represents an individual stock item with a stock name and the amount owned.
    """
    def __init__(self, stock, amt=1):
        """
        Initializes a stock item.

        :param stock: The name of the stock.
        :param amt: The amount of the stock owned (default is 1).
        """
        self.stock = stock
        self.amt = amt

class Stocks:
    """
    Represents a stock portfolio where users can buy and sell stocks using available funds.
    """
    def __init__(self, funds, stocks=None):
        """
        Initializes the Stocks portfolio.

        :param funds: Initial amount of funds available.
        :param stocks: Optional list of StockItem objects representing owned stocks.
        """
        self.funds = funds
        self.stocks = stocks if stocks is not None else []
        self.value  = 0

    def buy(self, stock, price):
        """
        Purchases a stock if sufficient funds are available.

        :param stock: The name of the stock to buy.
        :param price: The price of the stock.
        """
        if self.funds < price:
            print(f"Insufficient funds to buy {stock}. Available: ${self.funds:.2f}, Required: ${price:.2f}")
            return

        self.funds -= price  # Deduct the stock price from funds
        found = False

        # Check if the stock is already owned
        for el in self.stocks:
            if el.stock == stock:
                el.amt += 1  # Increase the stock amount
                found = True
                break

        # If stock is not found, add a new entry
        if not found:
            self.stocks.append(StockItem(stock))

        print(f"Bought 1 share of {stock} for ${price:.2f}. Remaining funds: ${self.funds:.2f}")

    def sell(self, stock, price):
        """
        Sells a stock if the user owns it.

        :param stock: The name of the stock to sell.
        :param price: The price of the stock at the time of selling.
        """
        for el in self.stocks:
            if el.stock == stock:
                el.amt -= 1  # Reduce stock amount
                self.funds += price  # Add the selling price to funds

                # Remove the stock from the list if the amount reaches zero
                if el.amt == 0:
                    self.stocks.remove(el)

                print(f"Sold 1 share of {stock} for ${price:.2f}. Updated funds: ${self.funds:.2f}")
                return

        print(f"Cannot sell {stock}: Not owned.")

    def has_stock(self, stock):
        """
        Checks if a stock exists in the portfolio.

        :param stock: The name of the stock to check.
        :return: True if the stock is in the portfolio, False otherwise.
        """
        return any(el.stock == stock for el in self.stocks)

    def percentage(self, stock):
        """
        Calculates the percentage of a specific stock in the portfolio.

        :param stock: The name of the stock to calculate percentage for.
        :return: The percentage of the stock in the portfolio, or 0 if not found or total is zero.
        """
        if not self.has_stock(stock):
            return 0.0

        total = sum(el.amt for el in self.stocks)  # Total number of shares
        if total == 0:  # Prevent division by zero
            return 0.0

        for el in self.stocks:
            if el.stock == stock:
                return el.amt / total
        return 0.0  # Fallback, though has_stock ensures this shouldn't happen

    def display_portfolio(self):
        """
        Displays the current stock holdings and available funds.
        """
        print(f"\nAvailable funds: ${self.funds:.2f}")
        if not self.stocks:
            print("Stock Portfolio is empty.")
        else:
            print("Stock Portfolio:")
            for stock in self.stocks:
                print(f"- {stock.stock}: {stock.amt} shares")
        print("-" * 30)


In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

def get_stock_prices(stocks, start_date, end_date):
    """
    Get historical prices for multiple stocks between two dates with automatic time frame selection.

    Args:
        stocks (list): List of dictionaries with stock info (must contain 'id' and 'symbol')
        start_date (str): Start date in 'YYYY-MM-DD' format
        end_date (str): End date in 'YYYY-MM-DD' format

    Returns:
        dict: Dictionary with stock symbols as keys and DataFrames of historical prices as values
    """
    # Convert string dates to datetime objects
    start = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(end_date, '%Y-%m-%d')

    # Calculate the time delta between start and end dates
    delta = end - start

    # Determine the appropriate interval based on the time range
    if delta <= timedelta(days=7):
        interval = '1m'  # 1 minute for intraday data (max 7 days)
    elif delta <= timedelta(days=60):
        interval = '5m'  # 5 minutes for short-term
    elif delta <= timedelta(days=365):
        interval = '1d'  # daily for medium-term
    else:
        interval = '1d'  # weekly for long-term

    results = {}

    for stock in stocks:
        try:
            # Download historical data
            data = yf.download(
                tickers=stock['symbol'],
                start=start_date,
                end=end_date,
                interval=interval,
                progress=False
            )

            # Add stock info to the results
            results[stock['symbol']] = {
                'prices': data,

            }

        except Exception as e:
            print(f"Error fetching data for {stock['symbol']}: {str(e)}")
            results[stock['id']] = {
                'symbol': stock['symbol'],
                'name': stock.get('name', stock['symbol']),
                'error': str(e)
            }

    return results


# Example usage:
if __name__ == "__main__":
    # Define your stocks (each with id and symbol)
    stocks = [
        {'id': 1, 'symbol': 'AAPL', 'name': 'Apple Inc.'},
        {'id': 2, 'symbol': 'MSFT', 'name': 'Microsoft Corporation'},

    ]
    stocks_predict = ["MSFT" , "AAPL" , "AMZN", "GOOGL"]

    # Define date range
    start_date = '2023-01-01'
    end_date = '2024-01-10'  # Will use 1-minute intervals for this 10-day period

    # Get the prices
    stock_prices = get_stock_prices(stocks, start_date, end_date)

    # Print results for the first stock
    first_stock_id = stocks[0]['symbol']
    print(stock_prices[ first_stock_id]["prices"]["Close"])
    close_prices = stock_prices[first_stock_id]["prices"]["Close"]

# Iterate over the Series using iteritems()
    for date, price in close_prices.iteritems():
      print(f"Date: {date}, Close Price: {price}")


Ticker            AAPL
Date                  
2023-01-03  123.632523
2023-01-04  124.907700
2023-01-05  123.583107
2023-01-06  128.130234
2023-01-09  128.654160
...                ...
2024-01-03  183.150375
2024-01-04  180.824356
2024-01-05  180.098694
2024-01-08  184.452560
2024-01-09  184.035065

[256 rows x 1 columns]


AttributeError: 'DataFrame' object has no attribute 'iteritems'

In [ ]:
def expansion(stocks, mainstock, portfolio, limit=0.2):
    """
    Expands the stock portfolio by evaluating whether to buy, sell, or hold each stock.

    :param stocks: List of stocks to evaluate.
    :param mainstock: The main stock object, which holds available funds.
    :param portfolio: The current portfolio that contains the user's owned stocks.
    :param limit: The threshold percentage limit for when a stock should be bought (default is 20%).
    :return: A list of actions to be taken (buy, sell, or hold) for each stock.
    """
    expand = []
    for stock in stocks:
        actions = {"hold": 0, "id": stock.id}  # Assuming stock has 'id' and 'price' attributes
        if portfolio.has_stock(stock.id):  # Check if the portfolio already has the stock
            actions["sell"] = stock.price  # Plan to sell the stock at its current price
        if portfolio.percentage(stock.id) < limit and mainstock.funds >= stock.price:
            actions["buy"] = -stock.price  # Plan to buy the stock if the limit is exceeded
        expand.append(actions)
    return expand




In [ ]:
def test_stocks():
    # Create a portfolio with $1000 initial funds
    portfolio = Stocks(funds=1000.00)

    # Test 1: Display initial empty portfolio
    print("Test 1: Initial Portfolio")
    portfolio.display_portfolio()

    # Test 2: Buy some stocks
    print("\nTest 2: Buying Stocks")
    portfolio.buy("AAPL", 150.50)  # Apple stock
    portfolio.buy("GOOG", 200.75)  # Google stock
    portfolio.buy("AAPL", 151.25)  # Buy more Apple stock
    portfolio.display_portfolio()

    try:
        aapl_percentage = portfolio.percentage("AAPL")
        print(aapl_percentage)

    except ZeroDivisionError:
        print("Error: Total stock amount is zero.")

    # Test 3: Check if stock exists
    print("\nTest 3: Checking Stock Existence")
    print(f"Has AAPL: {portfolio.has_stock('AAPL')}")
    print(f"Has TSLA: {portfolio.has_stock('TSLA')}")

    # Test 4: Sell stocks
    print("\nTest 4: Selling Stocks")
    portfolio.sell("AAPL", 155.00)  # Sell one Apple share
    portfolio.sell("GOOG", 205.50)  # Sell Google share
    portfolio.display_portfolio()

    # Test 5: Insufficient funds
    print("\nTest 5: Insufficient Funds")
    portfolio.buy("TSLA", 800.00)  # Try to buy Tesla with insufficient funds
    portfolio.display_portfolio()

    # Test 6: Sell non-owned stock
    print("\nTest 6: Sell Non-owned Stock")
    portfolio.sell("MSFT", 300.00)  # Try to sell Microsoft (not owned)
    portfolio.display_portfolio()

    # Test 7: Test percentage calculation
    print("\nTest 7: Percentage Calculation")
    portfolio.buy("AAPL", 152.00)  # Buy another Apple share
    portfolio.display_portfolio()
    try:
        aapl_percentage = portfolio.percentage("AAPL")
        print(aapl_percentage)

    except ZeroDivisionError:
        print("Error: Total stock amount is zero.")

# Run the test
if __name__ == "__main__":
    test_stocks()

Test 1: Initial Portfolio

Available funds: $1000.00
Stock Portfolio is empty.
------------------------------

Test 2: Buying Stocks
Bought 1 share of AAPL for $150.50. Remaining funds: $849.50
Bought 1 share of GOOG for $200.75. Remaining funds: $648.75
Bought 1 share of AAPL for $151.25. Remaining funds: $497.50

Available funds: $497.50
Stock Portfolio:
- AAPL: 2 shares
- GOOG: 1 shares
------------------------------
0.6666666666666666

Test 3: Checking Stock Existence
Has AAPL: True
Has TSLA: False

Test 4: Selling Stocks
Sold 1 share of AAPL for $155.00. Updated funds: $652.50
Sold 1 share of GOOG for $205.50. Updated funds: $858.00

Available funds: $858.00
Stock Portfolio:
- AAPL: 1 shares
------------------------------

Test 5: Insufficient Funds
Bought 1 share of TSLA for $800.00. Remaining funds: $58.00

Available funds: $58.00
Stock Portfolio:
- AAPL: 1 shares
- TSLA: 1 shares
------------------------------

Test 6: Sell Non-owned Stock
Cannot sell MSFT: Not owned.

Availabl

In [ ]:
portfolio = Stocks(10000)
portfolio.display_portfolio()


Available funds: $10000.00
Stock Portfolio is empty.
------------------------------


In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
from datetime import datetime

def calculate_financial_ratios(ticker, start_date, end_date, risk_free_rate=0.02, confidence_level=0.95):
    """
    Calculate financial ratios for a stock between two dates.

    Parameters:
    - ticker (str): Stock ticker symbol (e.g., "0700.HK" for Tencent)
    - start_date (str): Start date in "YYYY-MM-DD" format
    - end_date (str): End date in "YYYY-MM-DD" format
    - risk_free_rate (float): Annualized risk-free rate (default 2%)
    - confidence_level (float): Confidence level for VaR and CVaR (default 95%)

    Returns:
    - dict: Dictionary of calculated ratios
    """
    # Fetch stock data
    stock = yf.Ticker(ticker)
    stock_data = stock.history(start=start_date, end=end_date, interval="1d")

    # Fetch market data (Hang Seng Index as proxy)
    market = yf.Ticker("^HSI")
    market_data = market.history(start=start_date, end=end_date, interval="1d")

    # Calculate daily returns
    stock_returns = stock_data['Close'].pct_change().dropna()
    market_returns = market_data['Close'].pct_change().dropna()

    # Align dates (in case of missing data)
    common_dates = stock_returns.index.intersection(market_returns.index)
    stock_returns = stock_returns.loc[common_dates]
    market_returns = market_returns.loc[common_dates]

    # Annualization factor (252 trading days per year)
    trading_days = 252

    # 1. Standard Deviation (annualized)
    std_dev = stock_returns.std() * np.sqrt(trading_days)

    # 2. Value at Risk (VaR) - Parametric method
    mean_return = stock_returns.mean()
    var_percentile = np.percentile(stock_returns, 100 * (1 - confidence_level))
    var = var_percentile * np.sqrt(trading_days)  # Annualized

    # 3. Conditional Value at Risk (CVaR)
    cvar = stock_returns[stock_returns <= var_percentile].mean() * np.sqrt(trading_days)

    # 4. Sharpe Ratio
    excess_returns = stock_returns.mean() * trading_days - risk_free_rate
    sharpe_ratio = excess_returns / std_dev

    # 5. Sortino Ratio (only downside risk)
    downside_returns = stock_returns[stock_returns < 0]
    downside_std = downside_returns.std() * np.sqrt(trading_days)
    sortino_ratio = excess_returns / downside_std if downside_std != 0 else np.inf

    # 6. Treynor Ratio (requires beta)
    covariance = np.cov(stock_returns, market_returns)[0, 1]
    market_variance = market_returns.var()
    beta = covariance / market_variance if market_variance != 0 else np.nan
    treynor_ratio = excess_returns / beta if beta != 0 else np.inf

    # 7. Jensen’s Alpha
    market_excess = market_returns.mean() * trading_days - risk_free_rate
    jensen_alpha = (stock_returns.mean() * trading_days - risk_free_rate) - (beta * market_excess)

    # Return results as a dictionary
    return {
        "Standard Deviation": std_dev,
        "Value at Risk (VaR)": var,
        "Conditional Value at Risk (CVaR)": cvar,
        "Sharpe Ratio": sharpe_ratio,
        "Sortino Ratio": sortino_ratio,
        "Treynor Ratio": treynor_ratio,
        "Jensen’s Alpha": jensen_alpha,
        "Beta": beta  # Including beta for reference
    }

# Example usage
ticker = "0700.HK"
start_date = "2023-01-01"
end_date = "2025-04-03"

ratios = calculate_financial_ratios(ticker, start_date, end_date)
for metric, value in ratios.items():
    print(f"{metric}: {value:.4f}")

Standard Deviation: 0.3390
Value at Risk (VaR): -0.4465
Conditional Value at Risk (CVaR): -0.6613
Sharpe Ratio: 0.7127
Sortino Ratio: 1.1501
Treynor Ratio: 0.2086
Jensen’s Alpha: 0.1563
Beta: 1.1582


In [ ]:
class Node:
    """
    Represents a node in a graph, typically used in search algorithms like A* or BFS.

    Attributes:
        cost (float): The cost associated with this node (e.g., stock price).
        heuristic (float): The heuristic value estimating the cost to reach the goal.
        id (str): A unique identifier for the node (e.g., stock ID).
        action (str): The action associated with this node (e.g., 'buy', 'sell', 'hold').
        neighbors (list): A list of neighboring nodes connected to this node.
    """
    def __init__(self, stock, heuristic, neighbors=None):
        """
        Initializes a Node object.

        :param stock: A dictionary or object containing stock details (id, action, price).
        :param heuristic: The heuristic value for the node.
        :param neighbors: A list of neighboring nodes (default is an empty list).
        """
        self.cost = stock.get("price", 0)  # Cost of the stock (default to 0 if not provided)
        self.heuristic = heuristic  # Heuristic value
        self.id = stock.get("id", "unknown")  # Stock ID (default to 'unknown' if not provided)
        self.action = stock.get("action", "none")  # Action (default to 'none' if not provided)
        self.neighbors = neighbors if neighbors is not None else []  # Neighbors list

    @property
    def total_cost(self):
        """
        Calculates the total cost (cost + heuristic) for algorithms like A*.

        :return: The sum of the cost and heuristic values.
        """
        return self.cost + self.heuristic

    def add_neighbor(self, neighbor):
        """
        Adds a neighboring node to this node's neighbors list.

        :param neighbor: The neighboring node to add.
        """
        if neighbor not in self.neighbors:
            self.neighbors.append(neighbor)

    def remove_neighbor(self, neighbor):
        """
        Removes a neighboring node from this node's neighbors list.

        :param neighbor: The neighboring node to remove.
        """
        if neighbor in self.neighbors:
            self.neighbors.remove(neighbor)

    def __repr__(self):
        """
        String representation of the node for debugging purposes.

        :return: A string summarizing the node's attributes.
        """
        return (
            f"Node(id={self.id}, action={self.action}, cost={self.cost:.2f}, "
            f"heuristic={self.heuristic:.2f}, total_cost={self.total_cost:.2f})"
        )

In [ ]:
def setNodes(expansion, heuristic=0):
    """
    Creates a graph of nodes based on the expansion list.

    :param expansion: A list of dictionaries representing the possible actions for each stock.
                      Each dictionary contains keys like 'id', 'buy', 'sell', 'hold', and optionally 'heuristic'.
    :param heuristic: The initial heuristic value for the base node (default is 0).
    :return: A dictionary mapping unique node identifiers to their respective Node objects.
    """
    # Create a base node to represent the starting point
    baseNode = Node(
        {"id": "emp", "action": "none", "price": 0},  # Base node has no ID, action, or cost
        heuristic,
        neighbors=[]  # Initially, no neighbors
    )

    # Dictionary to store all nodes, keyed by a unique identifier (e.g., "AAPL_buy")
    nodes = {"emp": baseNode}
    prev_nodes = [baseNode]  # Start with the base node as the only node in the previous layer

    # Iterate through the expansion list to create nodes for each stock and its actions
    for stock_info in expansion:
        stock_id = stock_info["id"]  # Get the stock ID
        stock_heuristic = stock_info.get("heuristic", heuristic)  # Use provided heuristic or default
        created_nodes = []  # List to store nodes created for this stock

        # Check for 'buy' action
        if "buy" in stock_info:
            buy_node = Node(
                {"id": stock_id, "action": "buy", "price": stock_info["buy"]},
                stock_heuristic,
                neighbors=[]
            )
            nodes[f"{stock_id}_buy"] = buy_node  # Unique key for the buy node
            created_nodes.append(buy_node)

        # Check for 'sell' action
        if "sell" in stock_info:
            sell_node = Node(
                {"id": stock_id, "action": "sell", "price": stock_info["sell"]},
                stock_heuristic,
                neighbors=[]
            )
            nodes[f"{stock_id}_sell"] = sell_node  # Unique key for the sell node
            created_nodes.append(sell_node)

        # Check for 'hold' action
        if "hold" in stock_info:
            hold_node = Node(
                {"id": stock_id, "action": "hold", "price": stock_info["hold"]},
                stock_heuristic,
                neighbors=[]
            )
            nodes[f"{stock_id}_hold"] = hold_node  # Unique key for the hold node
            created_nodes.append(hold_node)

        # Connect all nodes in the previous layer to all nodes created for this stock
        for prev_node in prev_nodes:
            for new_node in created_nodes:
                prev_node.neighbors.append(new_node)

        # Update prev_nodes to the newly created nodes for the next iteration
        prev_nodes = created_nodes

    return nodes

In [ ]:
import random
import statistics
from scipy.stats import norm
# Define the list of stock symbols (you can customize this list)
stock_symbols = [
    "AAPL", "MSFT", "AMZN", "GOOGL", "TSLA", "NVDA", "META", "NFLX",
    "AMD", "INTC", "PYPL", "ADBE", "CRM", "AVGO", "QCOM", "TXN",
    "CSCO", "IBM", "ORCL", "VRTX"
]

# Generate the stocks list with 20 stocks, each having 25 random prices
stocks = []
for symbol in stock_symbols:
    # Generate 25 random prices between 50 and 500 (adjust range as needed)
    prices = [round(random.uniform(50, 500), 2) * random.choice([1, -1]) for _ in range(50)]
    stocks.append({
        "symbol": symbol,
        "prices": prices
    })




def get_var_and_cvar(stocks, confidence_level=0.95):
    """
    Calculate Value at Risk (VaR) and Conditional Value at Risk (CVaR) for each stock.

    Args:
        stocks (list): List of dictionaries containing 'symbol' and 'prices'.
        confidence_level (float): Confidence level for VaR (e.g., 0.95 for 95%).

    Returns:
        dict: Dictionary with stock symbols as keys and their VaR/CVaR as values.
    """
    ratios = {}
    z_score = norm.ppf(1 - confidence_level)  # Calculate Z-score for the given confidence level

    for stock in stocks:
        prices = stock["prices"]

        # Calculate VaR based on mean and standard deviation
        std_dev = statistics.stdev(prices)
        mean = statistics.mean(prices)
        var = mean - z_score * std_dev

        # Calculate drawdowns (losses relative to the highest price)
        max_value = max(prices)
        drawdowns = [((max_value - price) / max_value) for price in prices if ((max_value - price) / max_value) > 0]

        # Calculate CVaR (average of worst-case drawdowns)
        if drawdowns:
            cvar = sum(drawdowns) / len(drawdowns)
        else:
            cvar = 0  # No drawdowns, so CVaR is 0

        # Store results
        ratios[stock['symbol']] = {
            "VaR": var,
            "CVaR": cvar
        }

    return ratios
print(get_var_and_cvar(stocks))

{'AAPL': {'VaR': np.float64(519.322702448413), 'CVaR': 0.9228445823248957}, 'MSFT': {'VaR': np.float64(560.3572899606911), 'CVaR': 0.909841597880864}, 'AMZN': {'VaR': np.float64(452.0198267842408), 'CVaR': 1.1593142824388254}, 'GOOGL': {'VaR': np.float64(376.81214900587497), 'CVaR': 1.2853375090840276}, 'TSLA': {'VaR': np.float64(589.3703358291303), 'CVaR': 0.9337020774346265}, 'NVDA': {'VaR': np.float64(526.4697114660474), 'CVaR': 0.9406598062388266}, 'META': {'VaR': np.float64(498.19082149934), 'CVaR': 0.9970151631766476}, 'NFLX': {'VaR': np.float64(579.8829198100423), 'CVaR': 0.940731120478355}, 'AMD': {'VaR': np.float64(490.1421218086561), 'CVaR': 1.0685195347432102}, 'INTC': {'VaR': np.float64(615.1983052167986), 'CVaR': 0.8567562671629073}, 'PYPL': {'VaR': np.float64(471.0566229656634), 'CVaR': 1.0590067723975318}, 'ADBE': {'VaR': np.float64(500.7341068965334), 'CVaR': 1.0323260325991097}, 'CRM': {'VaR': np.float64(531.8161408125701), 'CVaR': 0.9084512990660439}, 'AVGO': {'VaR': 

In [ ]:
from math import sqrt
import random
import statistics
from scipy.stats import norm
# Define the list of stock symbols (you can customize this list)
stock_symbols = [
    "AAPL", "MSFT", "AMZN", "GOOGL", "TSLA", "NVDA", "META", "NFLX",
    "AMD", "INTC", "PYPL", "ADBE", "CRM", "AVGO", "QCOM", "TXN",
    "CSCO", "IBM", "ORCL", "VRTX"
]

# Generate the stocks list with 20 stocks, each having 25 random prices
stocks = []
for symbol in stock_symbols:
    # Generate 25 random prices between 50 and 500 (adjust range as needed)
    prices = [round(random.uniform(50, 500), 2) * random.choice([1, -1]) for _ in range(50)]
    stocks.append({
        "symbol": symbol,
        "prices": prices
    })


def get_Treyno(prices , risc_free = 0.02):
    returns = [((prices[i] - prices[i+1] )/(prices[i]))for i in range(len(prices) -1)]
    ret = sum(returns)
    std = statistics.stdev(returns)
    num = len(prices)
    beta = std * sqrt(num)
    return (ret - risc_free) / beta
